In [1]:
import pandas as pd
from pathlib import Path

from signlens.params import *
from signlens.preprocessing import data, preprocess
from utils import plot_landmarks


# reload automatically python functions outside notebook
%load_ext autoreload
%autoreload 2

2024-03-21 15:48:15.302801: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-21 15:48:18.101711: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Data from landmarks

In [2]:
data_landmarks = data.load_data_subset_csv(frac=1, n_classes=250)

✅ File with frames already exists, loaded matching 'sequence_id' rows.
✅ Filtered on n_frames = 100. Size reduced from 94477 to 86168 (91.2%)
✅ Filtered on n_classes = 250. Size reduced from 86168 to 86168 (100.0%)
✅ Loaded 86168 rows (91.2% of the original 94477 rows) from the dataset.


In [3]:
gloss_landmarks = pd.DataFrame(data_landmarks['sign'].unique())\
            .rename(columns={0:'sign'})
gloss_landmarks['sign'] = gloss_landmarks['sign'].str.lower()
gloss_landmarks.sort_values(by='sign', inplace=True, ignore_index=True)
gloss_landmarks


,sign
0,after
1,airplane
2,all
3,alligator
4,animal
...,...
245,yesterday
246,yourself
247,yucky
248,zebra


## Data from videos

In [4]:
WLASL_DIR = Path('../../raw_data/WLASL')
WLASL_JSON_PATH = WLASL_DIR/'WLASL_v0.3.json'
WLASL_VIDEO_DIR = WLASL_DIR/'videos'

data_videos = pd.read_json(WLASL_JSON_PATH)

In [5]:
gloss_videos = pd.DataFrame(data_videos['gloss'].unique())\
            .rename(columns={0:'sign_videos'})
gloss_videos['sign_videos'] = gloss_videos['sign_videos'].str.lower()
gloss_videos


,sign_videos
0,book
1,drink
2,computer
3,before
4,chair
...,...
1995,washington
1996,waterfall
1997,weigh
1998,wheelchair


## Merge

In [6]:
# Merge df with videos and df from landmarks to take only the matchin elements (the sign videos should be sorted by most used asl signs)
merged_df = gloss_videos.merge(gloss_landmarks, left_on='sign_videos', right_on='sign')
merged_df

,sign_videos,sign
0,book,book
1,drink,drink
2,before,before
3,chair,chair
4,go,go
...,...,...
196,gift,gift
197,lamp,lamp
198,penny,penny
199,stuck,stuck


In [7]:
most_used_signs = merged_df.sign
most_used_signs

0        book
1       drink
2      before
3       chair
4          go
        ...  
196      gift
197      lamp
198     penny
199     stuck
200        tv
Name: sign, Length: 201, dtype: object

In [17]:
# take the missing signs from the landmarks glossary to append them
missing_signs = gloss_landmarks[~gloss_landmarks.sign.isin(most_used_signs)]
missing_signs.head()

,sign
11,backyard
33,callonphone
40,cheek
42,chin
50,cowboy


In [18]:
missing_signs.shape

(49, 1)

In [19]:
all_signs = pd.concat([most_used_signs, missing_signs])
all_signs

,sign
0,book
1,drink
2,before
3,chair
4,go
...,...
233,wake
236,weus
247,yucky
248,zebra


In [24]:
glossary_csv_path = TRAIN_DATA_DIR + '/glossary.csv'
glossary_csv_path 

'/home/bfrisque/code/benoitfrisque/signlens/raw_data/asl-signs/glossary.csv'

In [27]:
all_signs.to_csv(csv_path, index=False)

In [28]:
pd.read_csv(csv_path)

,sign
0,book
1,drink
2,before
3,chair
4,go
...,...
245,wake
246,weus
247,yucky
248,zebra
